# Install HuggingFace dependencies

In [1]:
!pip install transformers
!pip install tqdm
!pip install bertviz
!pip install segtok

     |████████████████████████████████| 2.1MB 6.3MB/s 
     |████████████████████████████████| 3.3MB 20.4MB/s 
     |████████████████████████████████| 901kB 42.1MB/s 
     |████████████████████████████████| 163kB 5.2MB/s 
     |████████████████████████████████| 1.2MB 8.9MB/s 
     |████████████████████████████████| 133kB 16.7MB/s 
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 7.4MB 16.7MB/s 
ERROR: botocore 1.20.59 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
  Created wheel for segtok: filename=segtok-1.5.10-cp37-none-any.whl size=25019 sha256=feec9e5ba9eea9f26dfbb9448906d24decad75dbb66800284f63ffe50d908649
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
Successfully built segtok


# Mount Drive and Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys, os, json

In [ ]:
root_folder = "/content/drive/My Drive/cs182_project/"
output_dir = os.path.join(root_folder, "./model_save/")
data_path = os.path.join(root_folder, "yelp_review_training_dataset.jsonl")
model_path = os.path.join(root_folder, "model_experiment.pt")

In [ ]:
sys.path.append(root_folder)

# Load Finetuned BERT Model

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

from tqdm import tqdm

from model import ReviewPredictionModel

In [ ]:
model = ReviewPredictionModel(0, 128)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
model.eval()
bert = model.transfomer
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
# config = GPT2Config.from_pretrained(output_dir)
# model = GPT2LMHeadModel.from_pretrained(output_dir)
# tokenizer = GPT2Tokenizer.from_pretrained(output_dir)

In [ ]:
if torch.cuda.is_available():
  device = "cuda"
  model.cuda()
else:
  device = "cpu"
model.to(device)
print(device)

cpu


In [ ]:
from bertviz import head_view, model_view
from bertviz.transformers_neuron_view import BertTokenizer, BertModel
from bertviz.neuron_view import show

In [ ]:
prompt = "I absolutely loved this place with its splendid soup and salad"
prompt2 = 'Both were delicious'

In [ ]:
inputs = tokenizer.encode_plus(prompt, prompt2, return_tensors='pt', add_special_tokens=True, return_token_type_ids=True)
inputs.to(device)
token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
attention = bert(input_ids, output_attentions=True)[-1]

In [ ]:
input_id_list = input_ids[0].tolist() # Batch index 0
sentence_b_start = token_type_ids[0].tolist().index(1)
tokens = tokenizer.convert_ids_to_tokens(input_id_list)

In [ ]:
head_view(attention, tokens, sentence_b_start)

<IPython.core.display.Javascript object>

In [ ]:
model_view(attention, tokens, sentence_b_start)

<IPython.core.display.Javascript object>

In [ ]:
bert_output_dir = './bert_model_save/'
bert_output_dir = os.path.join(root_folder, "./bert_model_save/")
if False:
  

  # Create output directory if needed
  if not os.path.exists(bert_output_dir):
      os.makedirs(bert_output_dir)

  print("Saving model to %s" % bert_output_dir)

  # Save a trained model, configuration and tokenizer using `save_pretrained()`.
  # They can then be reloaded using `from_pretrained()`
  model_to_save = bert.module if hasattr(bert, 'module') else bert  # Take care of distributed/parallel training
  model_to_save.save_pretrained(bert_output_dir)
  tokenizer.save_pretrained(bert_output_dir)
  bert.config.save_pretrained(bert_output_dir)

In [ ]:
bert_saved = BertModel.from_pretrained(bert_output_dir, output_attentions=True)
bert_tokenizer = BertTokenizer.from_pretrained(bert_output_dir, output_attentions=True)

In [ ]:
show(bert_saved, 'bert', bert_tokenizer, prompt, prompt, layer=2, head=0)